In [ ]:
def UprightState():
    return np.array([0, np.pi, 0, 0])

def cart_pole_sim(use_new, num_samples, plotting):
    # defining the controller

    x = Variable("x")
    theta = Variable("t")
    xdot = Variable("\dot{x}")
    thetadot = Variable("\dot{t}")
    u_symbolic = Variable("u")
    garbage = Variable("garbage")

    x = np.hstack([x, theta, xdot, thetadot])
    u = -K@(x  - UprightState())
    own_controller = SymbolicVectorSystem(state=[garbage], input=x, dynamics=np.array([0]), output=u)
    
    M = np.array([[1, 0, 0, 0], 
             [0, 1, 0, 0], 
             [0, 0, mc+mp, l*mp*pf.cos(theta)], 
             [0, 0, l*mp*pf.cos(theta), l**2*mp]])
    K2 = B.T@X@M
    u2 = -K2@(x - UprightState())
    own_controller_new = SymbolicVectorSystem(state=[garbage], input=x, dynamics=np.array([0]), output=u2)

    #defining the system
    exp_dynamics = np.array([xdot, 
                             thetadot, 
                             (1/(mc+mp*(pf.sin(theta))**2))*(u_symbolic + \
                                                mp*pf.sin(theta)*(l*thetadot**2+gravity*pf.cos(theta))),
                             (1/(l*mc+l*mp*(pf.sin(theta))**2))*(-u_symbolic*pf.cos(theta) \
                                -mp*l*thetadot**2*pf.cos(theta)*pf.sin(theta)-(mc+mp)*gravity*pf.sin(theta))])
    system = SymbolicVectorSystem(state=x, input=[u_symbolic], dynamics=exp_dynamics, output=x)



    builder = DiagramBuilder()
    # plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.1)
    # file_name = FindResource("models/cartpole.urdf")
    # Parser(plant).AddModelFromFile(file_name)
    # plant.Finalize()
    # print('Plant Finalised')

    plant = builder.AddSystem(system)
    
    if use_new: 
        controller = builder.AddSystem(own_controller_new)
    else: 
        controller = builder.AddSystem(own_controller)
    builder.Connect(plant.get_output_port(), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0),
                    plant.get_input_port())

    logger = LogVectorOutput(plant.get_output_port(), builder)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    #plant_context = plant.GetMyMutableContextFromRoot(context)

    # Simulate
    simulator.set_target_realtime_rate(0)
    duration = 50
    

    XR = np.linspace(-70, 70, num_samples)
    TR = np.linspace(1, 5, num_samples)
    XR1, TR1 = np.meshgrid(XR, TR)

    success_list = np.zeros_like(XR1)

    for i in range(len(XR1)):
        for j in range(len(XR1[i])): 

            context.SetTime(0.)

            trial = np.array([XR1[i, j], TR1[i, j], 0, 0, 0])
            context.SetContinuousState(trial)
            simulator.Initialize()
            simulator.AdvanceTo(duration)

            log = logger.FindLog(context)
            end_state = log.data()[:, -1]
            diff = np.abs(end_state - UprightState())

            tolerance = 5e-2

            if np.sum(diff < np.ones(4)*tolerance) == 4: 
                success_list[i, j] = 1


            if plotting: 
                plt.figure(figsize=(5, 4))
                plt.plot(log.sample_times(), log.data().transpose())
                plt.xlabel('t')
                plt.ylabel('y(t)')
            print(j)
        
        print(i)
            
    return success_list, TR1, XR1

print('HERE')
init_time = time.time()
success_list, TR1, XR1 = cart_pole_sim(False, 200, False)
np.save('cartpole300by300_expanded.npy', success_list)
end_time = time.time() - init_time
print(end_time)
success_list, TR1, XR1 = cart_pole_sim(True, 200, False)
np.save('cartpole300by300_new_expanded.npy', success_list)
end_time = time.time() - end_time